# ProxyWhirl Examples

Hands-on, runnable examples covering the Python library, CLI, and REST API. Use this as a living notebook while exploring the project.

_Updated: February 2026_

## Table of Contents
- [0. Setup](#0-setup)
- [1. Core Library Quickstart](#1-core-library-quickstart)
- [2. Strategy Deep Dive](#2-strategy-deep-dive)
- [3. Fetching & Validation](#3-fetching--validation)
- [4. Persistence Backends](#4-persistence-backends)
- [5. CLI Workbench](#5-cli-workbench)
- [6. REST API Walkthrough](#6-rest-api-walkthrough)
- [7. Advanced Patterns](#7-advanced-patterns)
- [8. Where to Go Next](#8-where-to-go-next)

## 0. Setup
Install dependencies from the repo root (includes optional extras for storage + browser rendering):

```bash
uv sync --group dev --extra storage --extra js
# or, from PyPI
uv pip install "proxywhirl[storage,js]"
```

In [ ]:
import platform

import proxywhirl

print(f"Python {platform.python_version()} on {platform.system()}")
print(f"ProxyWhirl {proxywhirl.__version__}")

## 1. Core Library Quickstart
Rotate through a small pool with `ProxyRotator`, mocking outbound traffic so the example stays offline.

In [ ]:
import httpx
from unittest.mock import patch

from pydantic import SecretStr

from proxywhirl import Proxy, ProxyPool, ProxyRotator, ProxySource
from proxywhirl.strategies import RoundRobinStrategy

pool = ProxyPool(
    name="quickstart",
    proxies=[
        Proxy(url="http://proxy-us.local:8000", country_code="US", source=ProxySource.USER),
        Proxy(url="http://proxy-de.local:8000", country_code="DE", source=ProxySource.USER),
        Proxy(url="http://proxy-sg.local:8000", country_code="SG", source=ProxySource.USER),
    ],
)

pool.add_proxy(
    Proxy(
        url="http://secure-proxy.local:9000",
        username=SecretStr("demo-user"),
        password=SecretStr("demo-pass"),
        metadata={"tier": "paid"},
    )
)

rotator = ProxyRotator(
    proxies=[p.model_copy(deep=True) for p in pool.get_all_proxies()],
    strategy=RoundRobinStrategy(),
)

# Track which proxy the strategy picked so the mock can echo it
selection_order: list[str] = []
original_select = rotator.strategy.select

def select_with_tracking(pool, context=None):
    proxy = original_select(pool, context=context)
    selection_order.append(proxy.url)
    return proxy

rotator.strategy.select = select_with_tracking  # type: ignore[attr-defined]

def mock_handler(request: httpx.Request) -> httpx.Response:
    current_proxy = selection_order[-1] if selection_order else "unknown"
    payload = {
        "requested_url": str(request.url),
        "proxy_used": current_proxy,
    }
    return httpx.Response(200, json=payload, request=request)

_original_client = httpx.Client

def build_mock_client(*args, **kwargs):
    kwargs["transport"] = httpx.MockTransport(mock_handler)
    return _original_client(*args, **kwargs)

with patch("httpx.Client", build_mock_client):
    responses = [rotator.get("https://service.example/api/status").json() for _ in range(3)]

rotator.strategy.select = original_select  # Restore

print("Rotation order:", " -> ".join(selection_order))
print("Responses:")
for payload in responses:
    print(payload)

print("Pool stats:", rotator.get_pool_stats())

A quick maintenance pass to clear unhealthy proxies and inspect the pool composition.

In [ ]:
from proxywhirl import HealthStatus

scratch_pool = ProxyPool(
    name="maintenance",
    proxies=[p.model_copy(deep=True) for p in pool.get_all_proxies()],
)

scratch_pool.proxies[1].health_status = HealthStatus.DEAD
removed = scratch_pool.clear_unhealthy()
print(f"Removed {removed} unhealthy proxies; pool size now {scratch_pool.size}")
print("Source breakdown:", scratch_pool.get_source_breakdown())

## 2. Strategy Deep Dive
A side-by-side look at round-robin, random, weighted, sticky sessions, and geo-targeting.

In [ ]:
import random

from proxywhirl import SelectionContext, StrategyConfig
from proxywhirl.strategies import (
    GeoTargetedStrategy,
    RandomStrategy,
    RoundRobinStrategy,
    SessionPersistenceStrategy,
    WeightedStrategy,
)

random.seed(7)
strategy_pool = ProxyPool(
    name="strategy-demo",
    proxies=[
        Proxy(url="http://proxy-a.local:8000", country_code="US"),
        Proxy(url="http://proxy-b.local:8000", country_code="DE"),
        Proxy(url="http://proxy-c.local:8000", country_code="SG"),
    ],
)
for proxy in strategy_pool.proxies:
    proxy.health_status = HealthStatus.HEALTHY

rr = RoundRobinStrategy()
rr_order = [rr.select(strategy_pool).url for _ in range(3)]

rand = RandomStrategy()
random_picks = [rand.select(strategy_pool).url for _ in range(3)]

weighted = WeightedStrategy()
weighted.configure(
    StrategyConfig(weights={
        "http://proxy-a.local:8000": 5.0,
        "http://proxy-b.local:8000": 2.0,
        "http://proxy-c.local:8000": 1.0,
    })
)
weighted_picks = [weighted.select(strategy_pool).url for _ in range(5)]

session_strategy = SessionPersistenceStrategy()
sticky = [
    session_strategy.select(strategy_pool, SelectionContext(session_id="user-42")).url
    for _ in range(4)
]

geo_strategy = GeoTargetedStrategy()
geo_choice = geo_strategy.select(strategy_pool, SelectionContext(target_country="DE")).url

print("Round-robin order:", rr_order)
print("Random picks:", random_picks)
print("Weighted picks:", weighted_picks)
print("Sticky session (user-42):", sticky)
print("Geo-targeted (DE):", geo_choice)

## 3. Fetching & Validation
`ProxyFetcher` can parse multiple formats and validate entries. We patch `httpx.AsyncClient.get` so this cell stays offline.

In [ ]:
import asyncio
            import httpx
            from unittest.mock import patch

            from proxywhirl import Proxy, ProxySource
            from proxywhirl.fetchers import ProxyFetcher, ProxySourceConfig, ProxyValidator
            from proxywhirl.models import ProxyFormat, ValidationLevel

            sample_proxy_list = "
".join(
                [
                    "http://198.51.100.1:8080",
                    "http://198.51.100.2:8080",
                    "http://198.51.100.1:8080",
                ]
            )

            async def fetch_and_validate() -> None:
                fetcher = ProxyFetcher(
                    sources=[ProxySourceConfig(url="https://example.com/mock-proxies.txt", format=ProxyFormat.PLAIN_TEXT)],
                    validator=ProxyValidator(level=ValidationLevel.BASIC, concurrency=4),
                )

                async def fake_get(self, url, *args, **kwargs):
                    return httpx.Response(200, text=sample_proxy_list, request=httpx.Request("GET", url))

                with patch.object(httpx.AsyncClient, "get", new=fake_get):
                    raw_entries = await fetcher.fetch_all(validate=True)

                proxies = [Proxy(url=item["url"], source=ProxySource.FETCHED) for item in raw_entries]
                print(f"Fetched {len(raw_entries)} unique proxies")
                for proxy in proxies:
                    print(f"• {proxy.url} (source={proxy.source.value})")

            asyncio.run(fetch_and_validate())

## 4. Persistence Backends
File-based storage and SQLite both support async usage. Temporary directories keep the notebook tidy.

In [ ]:
import asyncio
import tempfile
from pathlib import Path

from proxywhirl import Proxy, ProxySource
from proxywhirl.storage import FileStorage

async def demo_file_storage() -> None:
    proxies = [
        Proxy(url="http://proxy-store-1.local:9000", source=ProxySource.USER),
        Proxy(url="http://proxy-store-2.local:9000", source=ProxySource.FETCHED),
    ]

    with tempfile.TemporaryDirectory() as tmpdir:
        storage_path = Path(tmpdir) / "proxies.json"
        storage = FileStorage(storage_path)
        await storage.save(proxies)
        loaded = await storage.load()
        print(f"Stored {len(loaded)} proxies at {storage_path}")
        for proxy in loaded:
            print(f"• {proxy.url} (source={proxy.source.value})")

asyncio.run(demo_file_storage())

In [ ]:
import asyncio
import tempfile
from pathlib import Path

from proxywhirl import HealthStatus, Proxy, ProxySource
from proxywhirl.storage import SQLiteStorage

async def demo_sqlite_storage() -> None:
    with tempfile.TemporaryDirectory() as tmpdir:
        db_path = Path(tmpdir) / "proxies.db"
        storage = SQLiteStorage(db_path)
        await storage.initialize()

        proxies = [
            Proxy(url="http://proxy-db-1.local:9000", source=ProxySource.USER, health_status=HealthStatus.HEALTHY),
            Proxy(url="http://proxy-db-2.local:9000", source=ProxySource.FETCHED, health_status=HealthStatus.DEGRADED),
        ]

        await storage.save(proxies)
        all_proxies = await storage.load()
        healthy_only = await storage.query(health_status=HealthStatus.HEALTHY.value)

        print(f"Database '{db_path.name}' contains {len(all_proxies)} proxies")
        print("Healthy query result:", [p.url for p in healthy_only])

        await storage.close()

asyncio.run(demo_sqlite_storage())

## 5. CLI Workbench
Run CLI commands directly from the notebook for quick exploration. Swap in `uv run proxywhirl ...` if you prefer uv-managed environments.

In [ ]:
import subprocess
            import sys


            def run_cli_command(args, output_lines=12):
                cmd = [sys.executable, "-m", "proxywhirl.cli"] + args
                print(f"$ proxywhirl {' '.join(args)}")
                result = subprocess.run(cmd, capture_output=True, text=True)
                if result.stdout:
                    lines = result.stdout.strip().splitlines()
                    preview = lines[:output_lines]
                    print("
".join(preview))
                    if len(lines) > output_lines:
                        print("…")
                if result.stderr:
                    print("[stderr]")
                    print(result.stderr.strip())
                print(f"(exit code: {result.returncode})
")

            run_cli_command(["--help"])
            run_cli_command(["--no-lock", "config", "show"], output_lines=15)
            run_cli_command(["--no-lock", "pool", "list"])

## 6. REST API Walkthrough
Spin up the FastAPI app in-process, add a proxy, and perform a proxied request. We patch `httpx.AsyncClient.request` to avoid real egress.

In [ ]:
try:
    from fastapi.testclient import TestClient
except ImportError as exc:  # pragma: no cover
    raise ImportError(
        "FastAPI is not installed. Install with `uv pip install 'proxywhirl[storage]'` "
        "to run this cell."
    ) from exc

import httpx
from unittest.mock import patch

from proxywhirl.api import app

async def mock_async_request(self, method, url, *args, **kwargs):
    return httpx.Response(200, json={"requested": url, "method": method}, request=httpx.Request(method, url))

with TestClient(app) as client:
    added = client.post("/api/v1/proxies", json={"url": "http://proxy-api.local:8000"})
    print("Add proxy ->", added.json())

    status_response = client.get("/api/v1/status").json()
    print("Status summary keys:", list(status_response["data"].keys()))

    with patch.object(httpx.AsyncClient, "request", new=mock_async_request):
        proxied = client.post(
            "/api/v1/request",
            json={"url": "https://example.org/json", "method": "GET", "timeout": 10},
        )
        print("Proxied request ->", proxied.json()["data"])  # type: ignore[index]

## 7. Advanced Patterns
- **Background health monitoring** — plug in your own check routine by subclassing `HealthMonitor` and overriding `_run_health_checks`.
- **Browser rendering** — add `proxywhirl[js]` and Playwright to scrape JS-heavy proxy catalogs with `BrowserRenderer`.
- **Built-in source catalogs** — explore `proxywhirl/sources.py` for 64+ free sources and `premium_sources.py` for vetted providers.

```python
# Health monitor skeleton
from proxywhirl import HealthMonitor, Proxy, ProxyPool

class CustomMonitor(HealthMonitor):
    async def _run_health_checks(self):
        for proxy in self.pool.get_all_proxies():
            self._record_success(proxy)  # replace with real checks
```

## 8. Where to Go Next
- Swap mock transports for real proxy endpoints and track metrics over time.
- Enable authentication and persistence for the REST API via environment variables.
- Wire the CLI into CI/CD with `--format json` for structured output.
- Browse the test suite in `tests/` for end-to-end patterns across rotation, retry, and failover.